##### %%writefile test
## Midterm
<b>The objective of the first part of the midterm is to detect blue cursor, yellow timer, and human skin from input video. Your output should be similar to midterm_demo.mp4. Please complete steps 1-8 in one single code cell. The whole process can be divided to the following steps:</b>
1. (5pts) Input images from video file WiiPlay.mp4 with the same level number as the last two digits of your student id, and show the images in the "input" window.
2. (5pts) Use <i>cv2.cvtColor()</i> to convert images from BGR to HSV format.
3. (5pts) Use <i>cv2.createTrackbar()</i> to create six trackbars (HueMin, HueMax, SatMin SatMax, ValMin, ValMax), and use <i>cv2.getTrackbarPos()</i> to get the current value of each trackbar.
4. (5pts) Use <i>cv2.threshold()</i> or <i>cv2.inRange()</i> to apply double thresholding to each channel (Hue, Sat, Val) based on current values of the six trackbars
5. (5pts) Apply morphological filters to remove noise (outliers & holes), and show the detected regions in the "test" window..
6. (10pts) Find out the best color range to detect <b>blue cursor</b>, apply these thresholds, and show the detected regions in the "cursor" window.
7. (10pts) Find out the best color range to detect <b>yellow timer</b>, apply these thresholds, and show the detected regions in the "timer" window.
8. (10pts) Find out the best color range to detect <b>human skin</b>, apply these thresholds, and show the detected regions in the "skin" window.



In [95]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img

In [100]:
def nothing(x):
    pass

cap = cv2.VideoCapture("WiiPlay.mp4")

if not cap.isOpened():
    raise IOError("Cannot open the video file")

frame_start = 150
frame_end = 437

cap.set(cv2.CAP_PROP_POS_FRAMES, frame_start)
cv2.namedWindow('input')
cv2.namedWindow('test')
cv2.namedWindow('cursor')
cv2.namedWindow('timer')
cv2.namedWindow('skin')

# 3. Use cv2.createTrackbar() to create six trackbars
cv2.createTrackbar('HueMin', 'test', 0, 30, nothing)
cv2.createTrackbar('HueMax', 'test', 0, 150, nothing)
cv2.createTrackbar('SatMin', 'test', 0, 10, nothing)
cv2.createTrackbar('SatMax', 'test', 0, 120, nothing)
cv2.createTrackbar('ValMin', 'test', 0, 50, nothing)
cv2.createTrackbar('ValMax', 'test', 0, 200, nothing)

cv2.createTrackbar('BlueHueMin', 'cursor', 100, 180, nothing)
cv2.createTrackbar('BlueHueMax', 'cursor', 120, 180, nothing)
cv2.createTrackbar('BlueSatMin', 'cursor', 100, 255, nothing)
cv2.createTrackbar('BlueSatMax', 'cursor', 255, 255, nothing)
cv2.createTrackbar('BlueValMin', 'cursor', 100, 255, nothing)
cv2.createTrackbar('BlueValMax', 'cursor', 255, 255, nothing)

cv2.createTrackbar('YellowHueMin', 'timer', 20, 30, nothing)
cv2.createTrackbar('YellowHueMax', 'timer', 30, 30, nothing)
cv2.createTrackbar('YellowSatMin', 'timer', 100, 255, nothing)
cv2.createTrackbar('YellowSatMax', 'timer', 255, 255, nothing)
cv2.createTrackbar('YellowValMin', 'timer', 100, 255, nothing)
cv2.createTrackbar('YellowValMax', 'timer', 255, 255, nothing)

cv2.createTrackbar('SkinHueMin', 'skin', 0, 30, nothing)
cv2.createTrackbar('SkinHueMax', 'skin', 20, 30, nothing)
cv2.createTrackbar('SkinSatMin', 'skin', 40, 255, nothing)
cv2.createTrackbar('SkinSatMax', 'skin', 255, 255, nothing)
cv2.createTrackbar('SkinValMin', 'skin', 50, 255, nothing)
cv2.createTrackbar('SkinValMax', 'skin', 255, 255, nothing)

current_frame = frame_start

while True:
    ret, frame = cap.read()

    if not ret:
        print("Reached end of video")
        break
    
    """ 
    1.Input images from video file WiiPlay.mp4 
    with the same level number as the last two digits of your student id, 
    and show the images in the "input" window
    """
    cv2.imshow('input', frame)

    # 2. Use cv2.cvtColor() to convert images from BGR to HSV format.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 3. use cv2.getTrackbarPos() to get the current value of each trackbar.
    hmin = cv2.getTrackbarPos('HueMin', 'test')
    hmax = cv2.getTrackbarPos('HueMax', 'test')
    smin = cv2.getTrackbarPos('SatMin', 'test')
    smax = cv2.getTrackbarPos('SatMax', 'test')
    vmin = cv2.getTrackbarPos('ValMin', 'test')
    vmax = cv2.getTrackbarPos('ValMax', 'test')

    blue_h_min = cv2.getTrackbarPos('BlueHueMin', 'cursor')
    blue_h_max = cv2.getTrackbarPos('BlueHueMax', 'cursor')
    blue_s_min = cv2.getTrackbarPos('BlueSatMin', 'cursor')
    blue_s_max = cv2.getTrackbarPos('BlueSatMax', 'cursor')
    blue_v_min = cv2.getTrackbarPos('BlueValMin', 'cursor')
    blue_v_max = cv2.getTrackbarPos('BlueValMax', 'cursor')

    # Get current trackbar positions for yellow timer detection
    yellow_h_min = cv2.getTrackbarPos('YellowHueMin', 'timer')
    yellow_h_max = cv2.getTrackbarPos('YellowHueMax', 'timer')
    yellow_s_min = cv2.getTrackbarPos('YellowSatMin', 'timer')
    yellow_s_max = cv2.getTrackbarPos('YellowSatMax', 'timer')
    yellow_v_min = cv2.getTrackbarPos('YellowValMin', 'timer')
    yellow_v_max = cv2.getTrackbarPos('YellowValMax', 'timer')

    skin_h_min = cv2.getTrackbarPos('SkinHueMin', 'skin')
    skin_h_max = cv2.getTrackbarPos('SkinHueMax', 'skin')
    skin_s_min = cv2.getTrackbarPos('SkinSatMin', 'skin')
    skin_s_max = cv2.getTrackbarPos('SkinSatMax', 'skin')
    skin_v_min = cv2.getTrackbarPos('SkinValMin', 'skin')
    skin_v_max = cv2.getTrackbarPos('SkinValMax', 'skin')


    lower = np.array([hmin, smin, vmin])
    upper = np.array([hmax, smax, vmax])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)

    lower_yellow = np.array([yellow_h_min, yellow_s_min, yellow_v_min])
    upper_yellow = np.array([yellow_h_max, yellow_s_max, yellow_v_max])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Define lower and upper bounds for blue cursor detection
    lower_blue = np.array([blue_h_min, blue_s_min, blue_v_min])
    upper_blue = np.array([blue_h_max, blue_s_max, blue_v_max])

    # Threshold the HSV image to get only blue cursor
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Define lower and upper bounds for skin detection
    lower_skin = np.array([skin_h_min, skin_s_min, skin_v_min])
    upper_skin = np.array([skin_h_max, skin_s_max, skin_v_max])

    # Threshold the HSV image to get only skin
    mask_skin = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    mask_blue = cv2.erode(mask_blue, kernel, iterations=1)
    mask_blue = cv2.dilate(mask_blue, kernel, iterations=1)

    mask_yellow = cv2.erode(mask_yellow, kernel, iterations=1)
    mask_yellow = cv2.dilate(mask_yellow, kernel, iterations=1)

    mask_skin = cv2.erode(mask_skin, kernel, iterations=1)
    mask_skin = cv2.dilate(mask_skin, kernel, iterations=1)

    cv2.imshow('input', frame)
    cv2.imshow('test', mask)
    cv2.imshow('cursor', mask_blue)
    cv2.imshow('timer', mask_yellow)
    cv2.imshow('skin', mask_skin)
    current_frame += 1

    if current_frame > frame_end:
        break

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


![input](input.png "input")
![test](test.png "test")
![cursor](cursor.png "cursor")
![timer](timer.png "timer")
![skin](skin.png "skin")

<b>The objective of the second part of the midterm is to stitch images together. Your output should be similar to the following picture. The coordinates of four pairs of corresponding points between two consecutive images should be provided <b style="color:red;">manually</b> (don't match automatically). Please complete steps 9-12 in one single code cell, steps 13-14 in another markdown cell, and upload your Jupyter notebook file (*.ipynb).</b>

9. (5pts) Given a pair of images (pier_1.jpg, pier_2.jpg), use <i>cv2.getPerspectiveTransform()</i> to get Homography Matrix by specifing the coordinates of four pairs of corresponding points <b>manually</b>.
10. (10pts) Use <i>cv2.warpPerspective()</i> and the Homography Matrix to get the projective view of the second image. You might need to make the output image larger enough to fit the projective image.
11. (10pts) Use <i>cv2.add()</i> to stitch two images together.
12. (10pts) Given a set of three images (pier_1.jpg, pier_2.jpg, pier_3.jpg) and their corresponding points, try to stitch them together.
13. (5pts) Any comments regarding the midterm? Which steps you believe you have completed? Which steps bother you?
14. (5pts) Any comments regarding the classes up to now? pace too fast or slow? quiz too hard or simple? prefer C or Python?

# Image Processing Report

## 1. Introduction

This report documents the process and methodology used to combine multiple images into a single canvas using perspective transformation techniques. The primary goal is to align and overlay images to create seamless transitions between them, utilizing OpenCV's functions for image manipulation.

## 2. Methodology

### Step 1: Read Input Images

First, the input images are loaded into the program using OpenCV's `imread` function.


In [101]:
img1 = cv2.imread("pier_1.jpg")
img2 = cv2.imread("pier_2.jpg")

### Step 2: Define Correspondence Points and Compute Homography Matrix

Points of correspondence between the images are defined to help in computing the homography matrix and use the defined points, a homography matrix is computed that maps points from the second image to the first.

In [102]:
pts_img1 = np.array([[387, 118], [401, 118], [421, 210], [390, 148]], dtype=np.float32)
pts_img2 = np.array([[99, 91], [113.5, 92], [132.5, 184.5], [101.5, 121]], dtype=np.float32)
#pts_img3 = np.array([[ 6.0 , 105.0 ] ,[ 67.0 , 105.5 ] , [ 66.5 , 231.5 ] ,[ 8.0 , 231.0 ]], dtype=np.float32)

Matrix = cv2.getPerspectiveTransform(pts_img2, pts_img1)

print("Homography Matrix:")
print(Matrix)

Homography Matrix:
[[ 7.72322265e-01  7.98423275e-02  2.92863937e+02]
 [-1.16594800e-01  1.00061083e+00  3.53130368e+01]
 [-4.13952260e-04  1.54733380e-04  1.00000000e+00]]


### Step 3: Warp Second Image

The second image is warped using the computed matrix, and its size is adjusted to fit the dimensions of the first image.

In [103]:
img1_height, img1_width = img1.shape[:2]
img2_height, img2_width = img2.shape[:2]

combine = cv2.warpPerspective(img2, Matrix, (img2_height * 2, img2_width* 2))

cv2.imshow("Combined", combine)
cv2.waitKey(0)

cv2.destroyAllWindows()

### Step 4: Overlay Images

The warped image is then overlayed onto a canvas, and blended with the first image to create a smooth transition.

In [104]:
combine_height, combine_width = combine.shape[:2]

canvas1 = np.zeros((combine_height + 100, combine_width + 100, 3), dtype=np.uint8)
canvas2 = np.zeros((combine_height + 100, combine_width + 100, 3), dtype=np.uint8)

canvas1[100:combine_height + 100, :combine_width] = combine
canvas2[100:img1_height + 100, :img1_width] = img1


### Step 5: Blending Images Using OpenCV

The given code snippet is used to blend two images based on their non-zero pixel values to create a composite image. Here's a breakdown of each step within the code:

#### 1: Initialize a Blank Canvas
A blank canvas (`canvas1_2`) is initialized with dimensions slightly larger than the combined images to accommodate both without any cropping. This canvas will hold the final blended image.

#### 2: Pixel-wise Blending Loop
A nested loop iterates over every pixel position in the new canvas. The conditions inside the loop determine how pixels from (`canvas1`) and (`canvas2`) are blended based on their values:

#### 3: Check and Blend Pixel Values
The conditions within the loop check the values of corresponding pixels in both (`canvas1`) and (`canvas2`)

(`1.Both Pixels Non-zero`)Both Pixels Non-zero: If both pixels are non-zero (i.e., they contain image data), their values are averaged. This averaging helps in merging overlapping parts of the images smoothly.

```python
if canvas1[i][j].all() != 0 and canvas2[i][j].all() != 0:
    canvas1_2[i][j] = (canvas1[i][j] + canvas2[i][j]) / 2
```

(`2.Only First Canvas Non-zero:`) If only the pixel from canvas1 contains image data, it is directly copied to the blended canvas. This condition preserves parts of the first image that do not overlap with the second.

```python
elif canvas1[i][j].all() != 0:
    canvas1_2[i][j] = canvas1[i][j]
```

(`3.Only Second Canvas Non-zero`)Similarly, if only the pixel from canvas2 is non-zero, it is copied to the blended canvas. This preserves parts of the second image in non-overlapping areas.

```python
elif canvas1[i][j].all() != 0:
    canvas1_2[i][j] = canvas1[i][j]
```

(`4.Both Pixels Zero`)If both pixels are zero, the corresponding pixel in the blended canvas remains zero (black), representing empty space.

```python
else:
    canvas1_2[i][j] = [0, 0, 0]
```

In [105]:
canvas1_2 = np.zeros(( combine_height+ 100, combine_width + 100, 3), dtype=np.uint8)

for i in range(combine_height + 100):

    for j in range(combine_width + 100):

        if canvas1[i][j].all() != 0 and canvas2[i][j].all() != 0:

            canvas1_2[i][j] = (canvas1[i][j] + canvas2[i][j]) / 2

        elif canvas1[i][j].all() != 0:

            canvas1_2[i][j] = canvas1[i][j]

        elif canvas2[i][j].all() != 0:

            canvas1_2[i][j] = canvas2[i][j]

        else:

            canvas1_2[i][j] = [0, 0, 0]

cv2.imwrite('pier_1&2.jpg', canvas1_2)
cv2.waitKey(0)

cv2.destroyAllWindows()


In [106]:
img3 = cv2.imread("pier_1&2.jpg")
img4 = cv2.imread("pier_3.jpg")

img3_height, img3_width = img3.shape[:2]
img4_height, img4_width = img4.shape[:2]

pts_img3 = np.float32([[681, 237], [679, 277], [651, 311], [775, 297]])
pts_img4 = np.float32([[67, 110], [66, 146], [44, 176], [143, 167]])

In [107]:
Matrix2 = cv2.getPerspectiveTransform(pts_img4, pts_img3)
print("Homography Matrix:")
print(Matrix2)

Homography Matrix:
[[ 1.22314346e+00  3.86178085e-02  5.98833400e+02]
 [-5.83784547e-02  1.14075979e+00  1.16830976e+02]
 [-5.54523051e-05  8.75996914e-05  1.00000000e+00]]


In [108]:
img3_resized = cv2.resize(img3, (img3_width, img3_height))

img3_height, img3_width = img3_resized.shape[:2]

combine2 = cv2.warpPerspective(img4, Matrix2, (img4_height * 3, img4_width * 3))
combine2_height, combine2_width = combine2.shape[:2]

canvas3 = np.zeros((combine2_height + 100, combine2_width + 100, 3), dtype=np.uint8)
canvas4 = np.zeros((combine2_height + 100, combine2_width + 100, 3), dtype=np.uint8)

canvas3[100:combine2_height + 100, :combine2_width] = combine2
canvas4[100:img3_height + 100, :img3_width] = img3_resized

canvas3_4 = np.zeros((combine2_height + 100, combine2_width + 100, 3), dtype=np.uint8)

In [109]:
for i in range(final_result_height + 100):
    
        for j in range(final_result_width + 100):
    
            if canvas3[i][j].all() != 0 and canvas4[i][j].all() != 0:
    
                canvas3_4[i][j] = (canvas3[i][j] + canvas4[i][j]) / 2
    
            elif canvas3[i][j].all() != 0:
    
                canvas3_4[i][j] = canvas3[i][j]
    
            elif canvas4[i][j].all() != 0:
    
                canvas3_4[i][j] = canvas4[i][j]
    
            else:
    
                canvas3_4[i][j] = [0, 0, 0]

cv2.imwrite('Result.jpg', canvas3_4)
cv2.waitKey(0)

cv2.destroyAllWindows()

![Result](Result.jpg "Result")

13. (5pts) Any comments regarding the midterm? Which steps you believe you have completed? Which steps bother you?
14. (5pts) Any comments regarding the classes up to now? pace too fast or slow? quiz too hard or simple? prefer C or Python?

13. For me, I finish all step. In my opinion I think that deal with data type is very difficult in Computer Vision, especially  when I try to combine the two images after calculate Matrix, The result always failed, however 

14. For me, I think not fast nor slow in class, maybe in final exam we can join a competition as our final score in Computer Vision, I prefer Python to do anything in Computer Vision